In [70]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
pal = sns.color_palette()
import pickle
from sklearn.svm import SVR
from sklearn.metrics import log_loss
import scipy
import math


from text_processor import TextProcessor

In [4]:
tp = TextProcessor()

## Load Doc2Vec

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin', binary=True)  

In [6]:
model.wv.most_similar(['woman'], topn=10)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('man', 0.7664012908935547),
 ('girl', 0.7494640946388245),
 ('teenage_girl', 0.7336829900741577),
 ('teenager', 0.631708562374115),
 ('lady', 0.6288785934448242),
 ('teenaged_girl', 0.6141783595085144),
 ('mother', 0.607630729675293),
 ('policewoman', 0.6069462299346924),
 ('boy', 0.5975908041000366),
 ('Woman', 0.5770983695983887)]

## Load Data

In [7]:
df_train = pd.read_csv('data/train.csv')
df_train = df_train.dropna() 
print("The training dataset has shape {}".format(df_train.shape))
df_train.head()

The training dataset has shape (404287, 6)


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [8]:
target = 'is_duplicate'
features = ['question1', 'question2']

X = df_train[features].as_matrix()
y = df_train[target].as_matrix()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Convert Data

In [71]:
def row_to_vec(row):
    """
    Return a [1, 2 * n_features] vector which is the concatenation of the word2vec vectors for the 
    two sentences (average of words)
    """
#     q1 = tp.clean(row[0])
#     q2 = tp.clean(row[1])
    q1 = tp.tokenize(row[0])
    q2 = tp.tokenize(row[1])
    
    q1_word_vectors = np.array([model.get_vector(word) for word in q1 if word in model.vocab])
    q1_vec = np.mean(q1_word_vectors, axis=0) if len(q1_word_vectors) > 0 else np.zeros((300,))
    
    q2_word_vectors = np.array([model.get_vector(word) for word in q2 if word in model.vocab])
    q2_vec = np.mean(q2_word_vectors, axis=0) if len(q2_word_vectors) > 0 else np.zeros((300,))
    
    
    result = scipy.spatial.distance.cosine(q1_vec, q2_vec)
    
    return 0 if math.isnan(result) else result

In [72]:
%%time
X_vec = np.apply_along_axis(row_to_vec, 1, X)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


CPU times: user 1min 50s, sys: 4 ms, total: 1min 50s
Wall time: 1min 50s


In [63]:
pickle.dump( {'X_vec': X_vec, 'y_vec': y}, open( "data/vec.dump", "wb" ) )

## SVM Classifier

In [73]:
y_vec = y

In [74]:
X_vec = X_vec.reshape((-1, 1))

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X_vec[:100000], y_vec[:100000], test_size=0.2, random_state=42)

In [81]:
np.isnan(X_train).any()

False

In [82]:
clf = SVR(C=1.0, epsilon=0.2)
clf.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [83]:
# from sklearn.naive_bayes import GaussianNB
# gnb = GaussianNB()
# gnb.fit(X_train, y_train)

In [84]:
y_pred = clf.predict(X_test)

In [85]:
log_loss(y_test, y_pred)

0.6150782784254385

In [47]:
X_test[0]

array([-3.46226171e-02,  1.72810882e-01,  2.16362849e-01,  1.57540217e-01,
       -1.20761447e-01, -1.18353948e-01, -6.60671666e-02, -2.49332011e-01,
        6.57569058e-03,  1.30706787e-01,  5.26767299e-02, -1.31174728e-01,
        1.70670405e-01, -6.73149973e-02, -8.79160613e-02,  1.50193959e-01,
        9.70052034e-02,  1.05501816e-01, -3.42305470e-03, -1.00865677e-01,
       -5.12661412e-02,  2.61623800e-01,  2.50142422e-02, -7.23089650e-02,
        6.33183569e-02, -9.33668315e-02, -2.42265284e-01,  1.75211579e-01,
        5.50467148e-02,  9.87514928e-02,  2.81914603e-02, -7.71840364e-02,
       -9.97721404e-02,  4.83487435e-02,  1.53913707e-01, -4.24728394e-02,
       -1.53852671e-01, -1.24448147e-02,  1.99066162e-01,  1.28343329e-03,
        4.65020090e-02, -7.25199357e-02, -1.09851412e-01,  1.56038068e-02,
       -9.27543640e-02, -9.26096141e-02,  1.97431780e-02,  4.74183820e-02,
        1.76018611e-01,  2.66018331e-01, -7.02243298e-03,  3.35726440e-01,
       -5.45662791e-02, -